### Libraries

In [1]:
# import libraries

import cv2
import os
import json
import numpy as np

### Data directory

In [2]:
# main paths

images_dir = './samples'
input = 'input.json'
output = 'output.json'

### Input file

In [27]:
# open an image

with open(input, 'r') as json_file:
    data = json.load(json_file)

image_paths = data['image_files']
image_index = 40

original_image = cv2.imread(os.path.join(images_dir, image_paths[image_index]))
original_image = cv2.resize(original_image, (0, 0), fx = 0.15, fy = 0.15)

cv2.imshow('Original', original_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [28]:
# gray and hsv versions

gray_image = cv2.cvtColor(original_image, cv2.COLOR_BGR2GRAY)

hsv_image = cv2.cvtColor(original_image, cv2.COLOR_BGR2HSV)

cv2.imshow('Gray', gray_image)
cv2.imshow('HSV', hsv_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

### Lego detection

In [29]:
image = original_image.copy()
gray = gray_image.copy()
hsv = hsv_image.copy()

hsv_flattened = hsv.reshape((-1, 3))
hsv_flattened = hsv_flattened[hsv_flattened[:, 2] > 0]
background_color = np.median(hsv_flattened, axis=0)

hue_tolerance = 105
saturation_tolerance = 105
value_tolerance = 105

lower_limit_0 = max(background_color[0] - hue_tolerance, 0)
upper_limit_0 = min(background_color[0] + hue_tolerance, 180)
lower_limit_1 = max(background_color[1] - saturation_tolerance, 0)
upper_limit_1 = min(background_color[1] + saturation_tolerance, 255)
lower_limit_2 = max(background_color[2] - value_tolerance, 0)
upper_limit_2 = min(background_color[2] + value_tolerance, 255)

lower_background = np.array([lower_limit_0, lower_limit_1, lower_limit_2])
upper_background = np.array([upper_limit_0, upper_limit_1, upper_limit_2])

hsv_blur = cv2.GaussianBlur(hsv, (11, 11), 0)
gray_mask = cv2.inRange(hsv_blur, lower_background, upper_background)
non_gray_mask = cv2.bitwise_not(gray_mask)

#clahe = cv2.createCLAHE(clipLimit=8.0, tileGridSize=(16, 16))
#clahe_image = clahe.apply(gray)
gray_blur = cv2.GaussianBlur(gray, (11, 11), 0)
canny = cv2.Canny(gray_blur, 50, 150)

combined_mask = cv2.bitwise_or(non_gray_mask, canny)
result = cv2.bitwise_and(image, image, mask=combined_mask)

contours, _ = cv2.findContours(combined_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

for contour in contours:
    x, y, w, h = cv2.boundingRect(contour)
    cv2.rectangle(image, (x, y), (x + w, y + h), (255, 0, 0), 2)

num_lego_pieces = len(contours)

cv2.imshow('Image', image)
cv2.imshow('Non-Gray Mask', non_gray_mask)
cv2.imshow('Canny', canny)
cv2.imshow('Combined Mask', combined_mask)
cv2.waitKey(0)
cv2.destroyAllWindows()

print("Number of LEGO pieces detected:", num_lego_pieces)

Number of LEGO pieces detected: 120


### Background medium testing

In [30]:
def update_tolerance(*args):
    global hue_tolerance, saturation_tolerance, value_tolerance
    hue_tolerance = cv2.getTrackbarPos("Hue", window_name)
    saturation_tolerance = cv2.getTrackbarPos("Saturation", window_name)
    value_tolerance = cv2.getTrackbarPos("Value", window_name)
    process_image()

def process_image():
    lower_limit_0 = max(background_color[0] - hue_tolerance, 0)
    upper_limit_0 = min(background_color[0] + hue_tolerance, 180)
    lower_limit_1 = max(background_color[1] - saturation_tolerance, 0)
    upper_limit_1 = min(background_color[1] + saturation_tolerance, 255)
    lower_limit_2 = max(background_color[2] - value_tolerance, 0)
    upper_limit_2 = min(background_color[2] + value_tolerance, 255)
    
    lower_background = np.array([lower_limit_0, lower_limit_1, lower_limit_2])
    upper_background = np.array([upper_limit_0, upper_limit_1, upper_limit_2])

    blur = cv2.GaussianBlur(hsv, (11, 11), 0)
    
    gray_mask = cv2.inRange(blur, lower_background, upper_background)
    non_gray_mask = cv2.bitwise_not(gray_mask)
    
    result = cv2.bitwise_and(image, image, mask=non_gray_mask)
    cv2.imshow(window_name, result)

image = original_image.copy()
hsv = hsv_image.copy()

hsv_flattened = hsv.reshape((-1, 3))
hsv_flattened = hsv_flattened[hsv_flattened[:, 2] > 0]
background_color = np.median(hsv_flattened, axis=0)

hue_tolerance = 100
saturation_tolerance = 100
value_tolerance = 100

window_name = "TEST"
cv2.namedWindow(window_name)

cv2.createTrackbar("Hue", window_name, hue_tolerance, 180, update_tolerance)
cv2.createTrackbar("Saturation", window_name, saturation_tolerance, 255, update_tolerance)
cv2.createTrackbar("Value", window_name, value_tolerance, 255, update_tolerance)

process_image()

cv2.waitKey(0)
cv2.destroyAllWindows()

error: OpenCV(4.9.0) D:\a\opencv-python\opencv-python\opencv\modules\highgui\src\window.cpp:866: error: (-215:Assertion failed) trackbar in function 'cv::getTrackbarPos'


error: OpenCV(4.9.0) D:\a\opencv-python\opencv-python\opencv\modules\highgui\src\window.cpp:866: error: (-215:Assertion failed) trackbar in function 'cv::getTrackbarPos'


error: OpenCV(4.9.0) D:\a\opencv-python\opencv-python\opencv\modules\core\src\arithm.cpp:1782: error: (-215:Assertion failed) lb.type() == ub.type() in function 'cv::inRange'


error: OpenCV(4.9.0) D:\a\opencv-python\opencv-python\opencv\modules\core\src\arithm.cpp:1782: error: (-215:Assertion failed) lb.type() == ub.type() in function 'cv::inRange'


error: OpenCV(4.9.0) D:\a\opencv-python\opencv-python\opencv\modules\core\src\arithm.cpp:1782: error: (-215:Assertion failed) lb.type() == ub.type() in function 'cv::inRange'


...

error: OpenCV(4.9.0) D:\a\opencv-python\opencv-python\opencv\modules\core\src\arithm.cpp:1782: error: (-215:Assertion failed) lb.type() == ub.type() in function 'cv::inRange'


error: OpenCV(4.9.0) D:\a\opencv-python\opencv-python\opencv\modules\core\src\arithm.cpp:1782: error: (-215:Assertion failed) lb.type() == ub.type() in function 'cv::inRange'


error: OpenCV(4.9.0) D:\a\opencv-python\opencv-python\opencv\modules\core\src\arithm.cpp:1782: error: (-215:Assertion failed) lb.type() == ub.type() in function 'cv::inRange'


error: OpenCV(4.9.0) D:\a\opencv-python\opencv-python\opencv\modules\core\src\arithm.cpp:1782: error: (-215:Assertion failed) lb.type() == ub.type() in function 'cv::inRange'


error: OpenCV(4.9.0) D:\a\opencv-python\opencv-python\opencv\modules\core\src\arithm.cpp:1782: error: (-215:Assertion failed) lb.type() == ub.type() in function 'cv::inRange'


### Canny testing

In [7]:
image = original_image.copy()
gray = gray_image.copy()
hsv = hsv_image.copy()

clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
clahe_image = clahe.apply(gray)

gaussian = cv2.GaussianBlur(clahe_image, (5, 5), 0)

canny = cv2.Canny(gaussian, 100, 200)

max_value = 255
threshold_low_value = 100
threshold_high_value = 200
window_name = "TEST"

cv2.namedWindow(window_name, cv2.WINDOW_AUTOSIZE)

def alter_low_value(*args):
    threshold_low_value = args[0]
    canny = cv2.Canny(gaussian, threshold_low_value, threshold_high_value)
    cv2.imshow(window_name, canny)

def alter_high_value(*args):
    threshold_high_value = args[0]
    canny = cv2.Canny(gaussian, threshold_low_value, threshold_high_value)
    cv2.imshow(window_name, canny)

cv2.createTrackbar("Low", window_name, threshold_low_value, max_value, alter_low_value)
cv2.createTrackbar("High", window_name, threshold_high_value, max_value, alter_high_value)

cv2.waitKey(0)
cv2.destroyAllWindows()